## Chapter 5 - Digital Building Blocks 



### 5.1 Introduction

### 5.2 Arithmetic Circuit 

### 5.3 Number Systems 

### 5.4 Sequential Building Blocks 

### 5.5 Memory Array 

### 5.6 Logic Array 

### 5.7 Summary 


## Exercises: